In [1]:
import datetime
import time
import pandas as pd
from sqlalchemy import create_engine
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
import time
from selenium.webdriver.support.wait import WebDriverWait
import selenium.webdriver.support.expected_conditions as EC
from selenium.webdriver.chrome.options import Options
from samoyan_pack.messenger1 import messenger1
import json
pd.set_option('display.max_columns', None)

# selenium 4
import random
import requests
from bs4 import BeautifulSoup

options = Options()
options.add_experimental_option(
    "prefs",
    {
        "download.default_directory": "./",
        "download.prompt_for_download": False,
        "download.directory_upgrade": True,
        "profile.default_content_setting_values.automatic_downloads": True,
    },
)
driver = webdriver.Chrome(options=options)
driver.get("https://www.12306.cn/index/")

In [38]:
def element_waiter(xpath: str):
    WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.XPATH, xpath)))


def tablize_order_page(driver):
    # Find the table element (adjust the locator as needed)
    table = driver.find_element(By.ID, "t-list")  # Example using XPath
    # Get the HTML content of the table
    table_html = table.get_attribute("outerHTML")
    # Use pandas to read the HTML table
    df = pd.read_html(table_html)[0]  # Extract the first table from the page

    # Extract IDs from table rows (adjust XPath based on your HTML structure)
    ids = []
    rows = table.find_elements(By.CLASS_NAME, "train")
    for row in rows:
        try:
            id_attr = row.get_attribute("id")  # Get the 'id' attribute of the row
            ids.append(id_attr)
        except:
            ids.append(None)  # Handle cases where 'id' attribute is not found

    # Add the 'id' column to the DataFrame
    df["id"] = ids
    return df


def list_to_dataframe(data_list):
    """Converts a list of pipe-delimited strings to a Pandas DataFrame.

    Args:
        data_list: A list of strings, where each string represents a row.

    Returns:
        A Pandas DataFrame, or None if the input is invalid.
    """
    if not data_list:  # Handle empty list case
        return pd.DataFrame()

    try:
        # Split each string into a list of values
        split_data = [row.split("|") for row in data_list]

        # Determine the number of columns (maximum number of splits)
        num_cols = max(len(row) for row in split_data)

        # Pad shorter rows with empty strings to ensure consistent columns
        for row in split_data:
            while len(row) < num_cols:
                row.append("")  # Or you could append None, np.nan, etc.

        # Create the DataFrame
        df = pd.DataFrame(split_data)
        return df
    except AttributeError:  # Handle cases where elements are not string
        print("Error: Input list contains non-string elements.")
        return None
    except Exception as e:
        print(f"An unexpected error occurred: {e}")
        return None

In [48]:
# deng lu hou -- zai ji xu

In [56]:
# if this code return something, that means you can browse the order page for your date now.,
# if not, the ticket is not yet start selling
try:
    find_date = '2025-01-24'
    headers = {
        'Accept': '*/*',
        'Accept-Encoding': 'gzip, deflate, br, zstd',
        'Accept-Language': 'en-US,en;q=0.9',
        'Cache-Control': 'no-cache',
        'Connection': 'keep-alive',
        'Cookie': f'_uab_collina=173528771562641942290566; JSESSIONID=458DB47EA2124C7EAE865B240E6DF358; tk=BVTefhDJca6p6BQ_hnElTlu8DZTAZzrejnf1f0; BIGipServerotn=1457062154.64545.0000; BIGipServerpassport=1005060362.50215.0000; guidesStatus=off; highContrastMode=defaltMode; cursorStatus=off; route=9036359bb8a8a461c164a04f8f50b252; uKey=77ef48a46ee28e6ee2249247327f65d738a826f392d5c93e568ff850850fe7fd; _jc_save_fromStation=%u4E0A%u6D77%2CSHH; _jc_save_toStation=%u592A%u539F%2CTYV; _jc_save_toDate=2024-12-27; _jc_save_wfdc_flag=dc; _jc_save_fromDate={find_date}'
    }
    r = requests.get(
        f"https://kyfw.12306.cn/otn/leftTicket/queryG?leftTicketDTO.train_date={find_date}&leftTicketDTO.from_station=SHH&leftTicketDTO.to_station=TYV&purpose_codes=ADULT"
        , headers = headers
    )
    rs = json.loads(r.content)
    rs = list_to_dataframe(rs['data']['result'])

    rs[[1,2,3,8,9,13]]
    # ticket_550000Z19607_01_06
except:
    print('ticket not yet start selling')
    driver.quit()

ticket not yet start selling


 qiang piao !!

In [22]:
t = 0
# driver.refresh()
while t < 999:
    # click the query button indefinately, to see the ticket is available or not.
    try:
        driver.find_element(By.ID, "query_ticket").click()
    except:
        pass
    """
    """
    time.sleep(random.randint(1, 15) / 10)
    t += 1
    # print(t)
    # this is the ticket I want.
    yuding_button_in_table_ = '//*[@id="ticket_550000Z19607_01_06"]/td[13]/a'
    
    try:
        driver.find_element(By.XPATH, yuding_button_in_table_).click()
        seattype_ = '//*[@id="seatType_1"]'
        element_waiter(seattype_)
        driver.find_element(By.XPATH, seattype_).click()
        try:
            # ruan wo huozhe ying wo
            driver.find_element(By.XPATH, '//*[@id="seatType_1"]/option[3]').click()
        except:
            driver.find_element(By.XPATH, '//*[@id="seatType_1"]/option[1]').click()
        # select passenger
        checkbox_select_passenger_ = '//*[@id="normal_passenger_id"]/li[1]/label'
        element_waiter(checkbox_select_passenger_)
        driver.find_element(By.XPATH, checkbox_select_passenger_).click()
        driver.find_element(By.XPATH, '//*[@id="submitOrder_id"]').click()
        driver.find_element(By.XPATH,'//*[@id="qr_submit_id"]').click()
    except:
        print("no find")
    break

no find


In [95]:
# driver.find_element(By.XPATH,'//*[@id="qr_submit_id"]').click()

In [23]:
def tablize_order_page(driver):
    # Find the table element (adjust the locator as needed)
    table = driver.find_element(By.ID, "t-list")  # Example using XPath
    # Get the HTML content of the table
    table_html = table.get_attribute("outerHTML")
    # Use pandas to read the HTML table
    df = pd.read_html(table_html)[0]  # Extract the first table from the page

    # Extract IDs from table rows (adjust XPath based on your HTML structure)
    ids = []
    rows = table.find_elements(By.CLASS_NAME, "train")
    for row in rows:
        try:
            id_attr = row.get_attribute("id")  # Get the 'id' attribute of the row
            ids.append(id_attr)
        except:
            ids.append(None)  # Handle cases where 'id' attribute is not found

    # Add the 'id' column to the DataFrame
    df["id"] = ids
    return df

In [24]:
df = tablize_order_page(driver)

In [25]:
df

,车次,出发站 到达站,出发时间到达时间,历时,商务座 特等座,优选 一等座,一等座,二等座 二等包座,高级 软卧,软卧/动卧 一等卧,硬卧 二等卧,软座,硬座,无座,其他,备注,id
0,G3144上海松江太原南06:4816:2509:37当日到达,G3144上海松江太原南06:4816:2509:37当日到达,G3144上海松江太原南06:4816:2509:37当日到达,G3144上海松江太原南06:4816:2509:37当日到达,*,--,*,*,--,--,--,--,--,--,--,14点30分起售,ticket_5q000G314400_01_23_train
1,G3144上海虹桥太原南07:1416:2509:11当日到达,G3144上海虹桥太原南07:1416:2509:11当日到达,G3144上海虹桥太原南07:1416:2509:11当日到达,G3144上海虹桥太原南07:1416:2509:11当日到达,*,--,*,*,--,--,--,--,--,--,--,13点30分起售,ticket_5q000G314400_02_23_train
2,G1952上海松江太原南07:4317:1009:27当日到达,G1952上海松江太原南07:4317:1009:27当日到达,G1952上海松江太原南07:4317:1009:27当日到达,G1952上海松江太原南07:4317:1009:27当日到达,*,--,*,*,--,--,--,--,--,--,--,14点30分起售,ticket_5q000G195207_03_18_train
3,G1952上海虹桥太原南08:1017:1009:00当日到达,G1952上海虹桥太原南08:1017:1009:00当日到达,G1952上海虹桥太原南08:1017:1009:00当日到达,G1952上海虹桥太原南08:1017:1009:00当日到达,*,--,*,*,--,--,--,--,--,--,--,13点30分起售,ticket_5q000G195207_04_18_train
4,G3132复上海虹桥太原南12:0220:5508:53当日到达,G3132复上海虹桥太原南12:0220:5508:53当日到达,G3132复上海虹桥太原南12:0220:5508:53当日到达,G3132复上海虹桥太原南12:0220:5508:53当日到达,*,--,*,*,--,--,--,--,--,--,--,13点30分起售,ticket_5j000G313200_08_26_train
5,G1956复上海虹桥太原南12:0321:2609:23当日到达,G1956复上海虹桥太原南12:0321:2609:23当日到达,G1956复上海虹桥太原南12:0321:2609:23当日到达,G1956复上海虹桥太原南12:0321:2609:23当日到达,*,--,*,*,--,--,--,--,--,--,--,13点30分起售,ticket_5l000G195602_01_20_train
6,G2407智复上海虹桥太原南12:0721:0809:01当日到达,G2407智复上海虹桥太原南12:0721:0809:01当日到达,G2407智复上海虹桥太原南12:0721:0809:01当日到达,G2407智复上海虹桥太原南12:0721:0809:01当日到达,*,--,*,*,--,--,--,--,--,--,--,13点30分起售,ticket_5l000G240700_01_18_train
7,G3207上海虹桥太原南12:2221:2008:58当日到达,G3207上海虹桥太原南12:2221:2008:58当日到达,G3207上海虹桥太原南12:2221:2008:58当日到达,G3207上海虹桥太原南12:2221:2008:58当日到达,*,--,*,*,--,--,--,--,--,--,--,13点30分起售,ticket_5l000G320700_01_17_train
8,G3207上海松江太原南12:4121:2008:39当日到达,G3207上海松江太原南12:4121:2008:39当日到达,G3207上海松江太原南12:4121:2008:39当日到达,G3207上海松江太原南12:4121:2008:39当日到达,*,--,*,*,--,--,--,--,--,--,--,14点30分起售,ticket_5l000G320700_02_17_train
9,K1806铺上海松江太原14:1111:2221:11次日到达,K1806铺上海松江太原14:1111:2221:11次日到达,K1806铺上海松江太原14:1111:2221:11次日到达,K1806铺上海松江太原14:1111:2221:11次日到达,--,--,--,--,--,*,*,--,*,*,--,14点30分起售,ticket_56000K180632_04_19_train
